In [1]:
d = 7
runs = 1000

from generate_matrices_tools import *


In [2]:
dysfunctional = generate_new_examples(runs, dimension = d, perimeter = 100, sensitivity = 1, jsonfilename = "dummyfile.json",  printing=True, functioning=False)

B: [ -97   69   78   44  100  -68  -67]
[  17   56   58  -60   24   78  -93]
[ -99  -35   44   88   91   -3    6]
[  93   45   63  -71 -100   61   14]
[  48  -94  -93  -64   77  -23   -6]
[  -2  -52   88   -1  -18  -97   88]
[ -21   65   85   85  -94  -70    5]
G: [ 41303   7426  23394 -19212 -11546   2326  11917]
[  7426  25698  -4979  13071  -5390 -13964  -5068]
[ 23394  -4979  29032 -23457  -4146   4983   2710]
[-19212  13071 -23457  33601 -10268    204   5492]
[-11546  -5390  -4146 -10268  30379  -3011 -26121]
[  2326 -13964   4983    204  -3011  27930  12979]
[ 11917  -5068   2710   5492 -26121  12979  32877]
lincomb_LLL: (1, -1, 1, 2, 2, -1, 1)
lincomb_cube: (1.0000, -1.3527, 0.61462, 1.7513, 0.68517, -0.77923, -0.062206)
sv_LLL: (50, -3, 1, 8, 45, -46, -35)
sv_cube: (17.782, 22.619, -0.65874, 6.5445, 20.968, -4.3398, 14.009)
lincomb_diff: (0.00000, 0.35266, 0.38538, 0.24869, 1.3148, -0.22077, 1.0622)
LLL.norm: 89.106689453125
cube.norm: 39.0615234375

B: [ 46  59   4 -89 -75 -83

In [3]:
print("There's {} dysfunctional matrices of dimension {} out of total {} instances, percent: {}". format(dysfunctional, d, runs, float(100*dysfunctional/runs)))


There's 288 dysfunctional matrices of dimension 7 out of total 1000 instances, percent: 28.8
